## Pobieranie danych: 
Dane z Polski nie mają gotowych paczek, które automatyzują pobieranie. Niemniej do zbierania treści służyć będzie kilka komend.   

Pracujemy na danych z systemu Ministerstwa Finansów - [Bestia@](https://bestia-api.mf.gov.pl/dev/index.html)

W pierwszej kolejności pobierzemy klasyfikację budżetową: 


In [ ]:
import requests
import json
import pandas as pd

# Tutaj mamy adres strony: 
URL = "https://bestia-api.mf.gov.pl" + "/api/dzialy?page=1&limit=25&format=JSON"

# Ta komenda pobiera zawartość adresu URL i następnie drukuje jej link
resp = requests.get(URL)
print(resp.url)

# Tu wydrukujemy odpowiedź serwera. Kod 200 oznacza, że pobraliśmy dane 
print(resp.status_code)

W zmiennej resp znajduje się zawartość odpowiedzi. To ramka danych typu JSON - analogiczna do tych, z którymi pracowaliśmy w Excelu. Będziemy chcieli ją przetworzyć:

In [ ]:
# dane trfiają do JSON, a następnie wybieram pole data
json_data = json.loads(resp.text)
firm_data = json_data["data"]

# Przekształcam wyniki do ramki danych Pandas
firm_df = pd.DataFrame(firm_data)

In [ ]:
print(firm_df)

## Przekazywanie parametrów przez słownik
Pierwsze nasze zapytanie miało bardzo długi kod opisujący parametry o które pytamy. Można to uprościć poprzez słownik. 

Zastosujemy go, aby zrobić łądną listę samorządów. 


In [ ]:
# Tutaj mamy adres strony: 
URL = "https://bestia-api.mf.gov.pl" + "/api/jednostki" 

# Ten słownik zbiera nasze informacje o parametrach zapytania
parametry_API = {
    'gt': 0, 
    'page': 1,
    'limit': 100,
    'format': "JSON"
    }
    
# Ta komenda pobiera zawartość adresu URL oraz dodaje informacje ze słownika 
resp = requests.get(URL, params =  parametry_API)

# Tu zobaczymy gotowy wydruk adresu
print(resp.url)

# Tu wydrukujemy odpowiedź serwera. Kod 200 oznacza, że pobraliśmy dane 
print(resp.status_code)

Przekształcenie danych z JSON do ramki danych jest takie same jak poprzednio:  

In [ ]:
# dane trfiają do JSON, a następnie wybieram pole data
json_data = json.loads(resp.text)
firm_data = json_data["data"]

# Przekształcam wyniki do ramki danych Pandas
firm_df = pd.DataFrame(firm_data)

Wydrukujemy tylko nazwy i klucz REGON:

In [ ]:
print(firm_df.loc[:,["nazwa","regon"]])

## Automatyzowanie zapytań
Analizy często wymagają zebrania setek, lub nawet tysiecy zapytań. Oczywiście robimy to w sposób zautomatyzowany korzystając z pętli. 

O czym trzeba pamiętać tworząc kod do takiego masowego pobierania: 
1.   Każdą uzyskaną odpowiedź warto od razu zapisywać na dysk - podczas komunikacji z obcym serwerem wiele rzeczy może pójść nie tak jak trzeba (np. straty połączenia z internetem) 
2.   Dlatego trzeba być gotowym, aby móc wznowić kod w dowolnym momencie pobierania. 

Wykorzystamy takie zautmatyzowane zapytanie, aby pobrać dane z formularza RS-27 dla Krakowa.

In [ ]:
# Tutaj mamy adres strony: 
URL = "https://bestia-api.mf.gov.pl" + "/api/pozycje-rb27s" 

# Ten słownik zbiera nasze informacje o parametrach zapytania
parametry_API = {
    'filter[jednostka-nazwa]': "krak", 
    'page': 1,
    'limit': 100,
    'format': "JSON"
    }

for i in range(1, 5):

    # Modyfikuje numer strony do wyszukania: 
    parametry_API['page'] = i

    resp = requests.get(URL, params =  parametry_API)

    # Tu zobaczymy gotowy wydruk adresu
    print("Zapytanie nr. ", i, ":", sep ="")
    print(resp.url)
    # Tu wydrukujemy odpowiedź serwera. Kod 200 oznacza, że pobraliśmy dane 
    print(resp.status_code)

    # dane trfiają do JSON, a następnie wybieram pole data
    json_data = json.loads(resp.text)
    firm_data = json_data["data"]

    # Przekształcam wyniki do ramki danych Pandas
    firm_df = pd.DataFrame(firm_data)
    
    # Zapisuje dane do pliku CSV
    firm_df.to_csv("Zapytanie"+str(i)+".csv")

## Dawkowanie zapytań

W poprzednim przykładzie wykonaliśmy 4 pytania. W standardowych warunkach chcemy zadać ich np. 7000 - tyle stron potrafią mieć kwartalne raporty samorządów. 

Automatyczne pytanie może w takich warunkach skończyć się banem ze strony ineresującego nas API. Aby tego uniknąć chcemy dodawać opóźnienia wykonania kodów. W Python służy do tego biblioteka *time* i funckja *sleep*  

In [ ]:
import time

for i in range(1, 5):
  print("Petla", i )
  time.sleep(10)

Porównajmy sobie działanie kodu bez funkcji sleep:

In [ ]:
for i in range(1, 5):
  print("Petla", i )

## Obsługa problemów:
Przygotowanie kodów wymaga brania pod uwagę sytuacji w których nie połączymy się z serwerem. 

W takim przypadku potrzebny będzie nam blok *try* i petla *while*.

To już zaczyna być nieco bardziej skomplikowane kodowanie: 

In [ ]:
import random

isConnected = False
attempt = 0 

while isConnected == False and attempt <= 15: 
  attempt += 1 
  try:
      #Tutaj rolę ryzykownej instrukcji gra sztuczne wygenerowanie dzielenia przez zero
      a = random.randint(0, 10)
      if a <= 8:
        b =  a / 0 
      isConnected = True
  except:
      print("Attempt ", attempt , " - interrupted")